In [1]:
from solve_LMI import solve_LMI
from solve_LP import solve_LP
import numpy as np
from scipy.linalg import toeplitz
from circulant_embedding import faster_A
from scipy.sparse import diags
from SCGAL import run_solver
import time
import json

In [ ]:
n = 1
max_runs = 100

starting_d = 21
ending_d = 25
ds = list(range(starting_d, ending_d+1))
circ_emb_obj = np.zeros((max_runs, len(ds)))
circ_emb_time = np.zeros((max_runs, len(ds)))
LMI_obj = np.zeros((max_runs, len(ds)))
LMI_time = np.zeros((max_runs, len(ds)))
SCGAL_obj = np.zeros((max_runs, len(ds)))
SCGAL_time = np.zeros((max_runs, len(ds)))
cs = np.random.normal(size=(100, 4*ds[-1], len(ds)))

# d = ds[0]
# c = cs[:4*d, 0]

In [3]:
# Circulant embedding
for i in range(len(ds)):
    d = ds[i]

    A = faster_A(n, d)
    A_0 = np.ones(np.shape(A)[0])

    start = time.perf_counter()
    total_time = 0
    num_iterations = 0
    while total_time < 60 and num_iterations < 100:
        c = cs[num_iterations, :4*d, i]
        tic = time.perf_counter()
        status_LP, optimal_value_LP, optimal_vars_LP = solve_LP(A_0, A, c, verbose=False)
        toc = time.perf_counter()
        time_of_this_iter = toc - tic
        total_time += time_of_this_iter
        print(d, total_time)
        circ_emb_obj[num_iterations, i] = optimal_value_LP
        circ_emb_time[num_iterations, i] = time_of_this_iter
        num_iterations += 1
    print(num_iterations)
        
        # print("LP Status:", status_LP)
        # print("LP Optimal Value:", optimal_value_LP)
        # print("LP Optimal Variables:", [var.value for var in optimal_vars_LP])

16 0.011051058012526482
16 0.01902787701692432
16 0.025585190975107253
16 0.03151688998332247
16 0.038338729995302856
16 0.044496164948213845
16 0.05053556594066322
16 0.05663487291894853
16 0.06261812889715657
16 0.06843129289336503
16 0.07588135689729825
16 0.082265768898651
16 0.08833361091092229
16 0.09427506290376186
16 0.09986303391633555
16 0.1058035139576532
16 0.11168002395424992
16 0.11734441295266151
16 0.1240634469431825
16 0.1301884539425373
16 0.13602566189365461
16 0.14271265891147777
16 0.14876450295560062
16 0.1548235339578241
16 0.16098996496293694
16 0.16665166296297684
16 0.17244880995713174
16 0.17818916600663215
16 0.1837369260028936
16 0.18943832302466035
16 0.19554383301874623
16 0.2013641339726746
16 0.2069601259427145
16 0.21267593192169443
16 0.2188162878737785
16 0.2246821858570911
16 0.23203174985246733
16 0.23859995882958174
16 0.24477157584624365
16 0.2521746978163719
16 0.25847993581555784
16 0.26487130380701274
16 0.2710133197833784
16 0.276876651798374

In [4]:
# LMI formulation with toeplitz constraints
for i in range(len(ds)):
    d = ds[i]

    col = np.zeros(2*d + 1, dtype=complex)
    col[0] = 1
    A_0 = toeplitz(col)

    A = []

    for m in range(1, 2*d + 1):
        col = np.zeros(2*d + 1, dtype=complex)
        col[m] = 1
        mx = toeplitz(col)
        A.append(mx)

    for m in range(1, 2*d + 1):
        col = np.zeros(2*d + 1, dtype=complex)
        col[m] = -1j
        mx = toeplitz(col)
        A.append(mx)

    start = time.perf_counter()
    total_time = 0
    num_iterations = 0
    while total_time < 60 and num_iterations < 100:
        c = cs[num_iterations, :4*d, i]
        tic = time.perf_counter()
        status_LMI, optimal_value_LMI, optimal_vars_LMI = solve_LMI(A_0, A, c, verbose=False)
        toc = time.perf_counter()
        time_of_this_iter = toc - tic
        total_time += time_of_this_iter
        print(d, total_time)
        LMI_obj[num_iterations, i] = optimal_value_LMI
        LMI_time[num_iterations, i] = time_of_this_iter
        num_iterations += 1
    print(num_iterations)

    # print("Toeplitz LMI Status:", status_LMI)
    # print("Toeplitz LMI Optimal Value:", optimal_value_LMI)
    # print("Toeplitz LMI Optimal Variables:", [var.value for var in optimal_vars_LMI])

16 0.1046674579847604
16 0.254036033002194
16 0.4203916970291175
16 0.5394932320341468
16 0.640484739036765
16 0.7594061380368657
16 0.8604790020617656
16 0.9415996010648087
16 1.1094354040105827
16 1.2180408550193533
16 1.3075023550190963
16 1.5330606970237568
16 1.643014186003711
16 1.7319760610116646
16 1.8230583139811642
16 1.9254725279752165
16 2.247642175992951
16 2.3570323929889128
16 2.4680407579871826
16 2.5861774099757895
16 2.666939790942706
16 2.7551471639308147
16 2.9196755909360945
16 3.0580963089014404
16 3.1436325219110586
16 3.2616987649234943
16 3.347677371930331
16 3.436326043913141
16 3.7437762349145487
16 3.8404196939081885
16 3.9367181629058905
16 4.053623017913196
16 4.141569603933021
16 4.236929675913416
16 4.972190114902332
16 5.060575712879654
16 5.194480353908148
16 5.286656233889516
16 5.43094854085939
16 5.556840201839805
16 5.643602636875585
16 5.760664585861377
16 5.898740169825032
16 5.985248167824466
16 8.822757266811095
16 8.956364859826863
16 9.078635

In [5]:
# SPARSE LMI formulation for Sketchy CGAL
for i in range(len(ds)):
    d = ds[i]

    diag = np.ones(2*d + 1, dtype=complex)
    A_0 = diags(diag)

    A = []

    for m in range(1, 2*d + 1):
        diag = np.ones(2*d + 1 - m, dtype=complex)
        mx = diags([diag, diag],offsets=[-m,m])
        A.append(mx)

    for m in range(1, 2*d + 1):
        diag = np.ones(2*d + 1 - m, dtype=complex)
        mx = diags([-1j*diag, 1j*diag],offsets=[-m,m])
        A.append(mx)

    A_norm = 2*d

    start = time.perf_counter()
    total_time = 0
    num_iterations = 0
    while total_time < 60 and num_iterations < 100:
        c = cs[num_iterations, :4*d, i]
        tic = time.perf_counter()
        U, Lambda, objective, Omega, z, y, S = run_solver(A_0, A, c, 2*d + 1, 4*d,
                                                        alpha=2*np.linalg.norm(c,ord=2), A_norm=A_norm,
                                                        R=1, T=5000,
                                                        trace_mode='bound')
        toc = time.perf_counter()
        time_of_this_iter = toc - tic
        total_time += time_of_this_iter
        print(d, total_time)
        SCGAL_obj[num_iterations, i] = -objective
        SCGAL_time[num_iterations, i] = time_of_this_iter
        num_iterations += 1
    print(num_iterations)

    # print("Calculated Objective:", -objective)

16 15.185404540039599
16 74.4591714850394
2
17 20.40677630796563
17 55.535866973979864
17 73.07025145500666
3
18 41.969785172026604
18 53.62823745101923
18 75.9205855090404
3
19 22.072064239007886
19 94.61180217296351
2
20 32.697798738023266
20 55.80460246902658
20 95.8523250920116
3


In [6]:
my_dict = {'ds': list(ds), 'circ_emb_obj': circ_emb_obj.tolist(), 'circ_emb_time': circ_emb_time.tolist(),
           'LMI_obj': LMI_obj.tolist(), 'LMI_time': LMI_time.tolist(),
           'SCGAL_obj': SCGAL_obj.tolist(), 'SCGAL_time': SCGAL_time.tolist()}
# Save the results to a JSON file
file_name = 'ds' + str(ds[0]) + 'to' + str(ds[-1]) + '.json'
with open(file_name, 'w') as f:
    json.dump(my_dict, f)

In [7]:
with open(file_name, 'r') as f:
    loaded_dict = json.load(f)

In [8]:
ds_loaded = np.array(loaded_dict['ds'])
circ_emb_obj_loaded = np.array(loaded_dict['circ_emb_obj'])
circ_emb_time_loaded = np.array(loaded_dict['circ_emb_time'])
LMI_obj_loaded = np.array(loaded_dict['LMI_obj'])
LMI_time_loaded = np.array(loaded_dict['LMI_time'])
SCGAL_obj_loaded = np.array(loaded_dict['SCGAL_obj'])
SCGAL_time_loaded = np.array(loaded_dict['SCGAL_time'])

In [9]:
assert np.allclose(ds, ds_loaded)
assert np.allclose(circ_emb_obj, circ_emb_obj_loaded)
assert np.allclose(circ_emb_time, circ_emb_time_loaded)
assert np.allclose(LMI_obj, LMI_obj_loaded)
assert np.allclose(LMI_time, LMI_time_loaded)
assert np.allclose(SCGAL_obj, SCGAL_obj_loaded)
assert np.allclose(SCGAL_time, SCGAL_time_loaded)

In [12]:
print(circ_emb_obj_loaded[0, 0])
print(LMI_obj_loaded[0, 0])
print(SCGAL_obj_loaded[0, 0])

-11.963613461378607
-15.171555146601715
-14.691614664834043


In [11]:
# start_idx = [1, 6, 11, 16]
# end_idx = [5, 10, 15, 20]

# for i in range(len(start_idx)):
#     file_name = 'ds' + str(start_idx[i]) + 'to' + str(end_idx[i]) + '.json'

#     with open(file_name, 'r') as f:
#         loaded_dict = json.load(f)

#     if i == 0:
#         ds_loaded = np.array(loaded_dict['ds'])
#         circ_emb_obj_loaded = np.array(loaded_dict['circ_emb_obj'])
#         circ_emb_time_loaded = np.array(loaded_dict['circ_emb_time'])
#         LMI_obj_loaded = np.array(loaded_dict['LMI_obj'])
#         LMI_time_loaded = np.array(loaded_dict['LMI_time'])
#         SCGAL_obj_loaded = np.array(loaded_dict['SCGAL_obj'])
#         SCGAL_time_loaded = np.array(loaded_dict['SCGAL_time'])
#     else:
#         ds_loaded = np.concatenate((ds_loaded, np.array(loaded_dict['ds'])), axis=0)
#         circ_emb_obj_loaded = np.concatenate((circ_emb_obj_loaded, np.array(loaded_dict['circ_emb_obj'])), axis=1)
#         circ_emb_time_loaded = np.concatenate((circ_emb_time_loaded, np.array(loaded_dict['circ_emb_time'])), axis=1)
#         LMI_obj_loaded = np.concatenate((LMI_obj_loaded, np.array(loaded_dict['LMI_obj'])), axis=1)
#         LMI_time_loaded = np.concatenate((LMI_time_loaded, np.array(loaded_dict['LMI_time'])), axis=1)
#         SCGAL_obj_loaded = np.concatenate((SCGAL_obj_loaded, np.array(loaded_dict['SCGAL_obj'])), axis=1)
#         SCGAL_time_loaded = np.concatenate((SCGAL_time_loaded, np.array(loaded_dict['SCGAL_time'])), axis=1)


# ds_loaded = ds_loaded[:17]
# circ_emb_obj_loaded = circ_emb_obj_loaded[:, :17]
# circ_emb_time_loaded = circ_emb_time_loaded[:, :17]
# LMI_obj_loaded = LMI_obj_loaded[:, :17]
# LMI_time_loaded = LMI_time_loaded[:, :17]
# SCGAL_obj_loaded = SCGAL_obj_loaded[:, :17]
# SCGAL_time_loaded = SCGAL_time_loaded[:, :17]
    